# Tutorial NN-CNN

![](neural.jpg)

# 1- ¿Qué son las redes neuronales?

Las redes neuronales son un tipo de modelo de machine learning que están diseñadas para operar de manera similar a las neuronas biológicas y el sistema nervioso humano. Estos modelos se usan para reconocer patrones complejos y relaciones que existen en un dataset con etiquetas. Tienen las siguientes propiedades:

    La arquitectura de un modelo de red neuronal está compuesta de un gran numero de nodos simples de procesamiento, llamados neuronas,  las cuales están interconectadas y organizadas en diferentes capas.

    Un nodo individual en una capa está conectado a muchos otros nodos de la capa anterior y de la capa siguiente. Las señales de entrada de una capa se reciben y procesan para generar la salida que se pasa a la siguiente capa.

    A la primera capa de esta arquitectura a menudo se la denomina capa de entrada, la cual recibe las señales de entrada, a la última capa se la llama capa de salida, la cual produce la señal de salida y al resto de capas que están entre la capa de entrada y la capa de salida se las llama capas ocultas.

Conceptos clave de las Redes Neuronales


**A. Neurona:**

Una neurona es una unidad de procesamiento de una red neuronal que está conectada a otras neuronas en la red. Esas conexiones representan las entradas y salidas de una neurona. Para cada una se estas conexiones, la neurona asigna un 'peso' (W-Weight) que es una medida de la importancia de la entrada y añade un término de sesgo (b-bias).



**B. Funciones de Activación:**

Las funciones de activación se usan para aplicar transformaciones no lineales en la entrada para lanzar la salida. El propósito de las funciones de activación es predecir la clase correcta de la variable objetivo basándose en las combinaciones de las variables de entrada. Algunas de las funciones de activación más populares son Relu, Sigmoide, y TanH.



**C. Propagación hacia Adelante (Forward Propagation):**

El modelo de red neuronal funciona con el proceso llamado propagación hacia delante en el cual se pasa la salida de activación en esa dirección sobre la red, de capa en capa.

Z = W*X + b
A = g(Z)

    g es la función de activación
    A es la activación usando la entrada
    W es el peso asociado con la entrada
    B es el sesgo asociado al nodo



**D. Error de Computación:**

La red neuronal aprende mejorando los valores de peso y sesgo. El modelo computa el error en la salida predicha en la última capa el cual es usado para hacer pequeños ajustes de los valores de los pesos y de los sesgos. Los ajustes se hacen de tal manera que el error total se minimiza. La función de pérdida, (Loss function), mide el error en la capa final y la función de coste, (Cost function), mide el error total de la red.

Pérdida = Valor_Actual - Valor_Predicho

Coste = Sumatoria (Pérdida)



**E. Propagación hacia Atrás (Backward Propagation):**

El modelo de red neuronal funciona con el proceso llamado propagación hacia atrás (backpropagation) en el cual el error se pasa hacia las capas anteriores de manera que esas capas también pueden mejorar el valor asociado a peso y sesgo. Usa el algoritmo llamado Gradiente Descente donde el error se minimiza y se obtienen los valores óptimos de los pesos y los sesgos. Este ajuste de los pesos y los sesgos se consigue computando el error de derivación, derivando los pesos, los sesgos y restandolos de los valores originales.





# 2-Implementando una red neuronal, clasificación binaria

Hagamos una sencilla implementación de una red neuronal en python para clasificación binaria, para clasificar si una imagen dada es 0 o 1.

In [1]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential
import pandas as pd 
import numpy as np 
import keras

Using TensorFlow backend.


**2.1 Preparación del dataset**

El primer paso es cargar y preparar el dataset.

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# incluye solo las filas con etiquetas = 0 o 1 (clasificación binaria)
X = train[train['label'].isin([0, 1])]

# variable objetivo
Y = train[train['label'].isin([0, 1])]['label']

# elimina la etiqueta de X
X = X.drop(['label'], axis = 1)

**2.2 Implementando una Función de Activación**

Usaremos la función de activación sigmoide porque su salida son valores entre 0 y 1 por lo que es una buena elección para un problema de clasificación binaria.

In [3]:
# implementando una sigmoide como función de activación
def sigmoide(z):
    s = 1.0/ (1 + np.exp(-z))    
    return s

**2.3 Se define la Arquitectura de la Red Neuronal**

Se crea un modelo con tres capas - Entrada, Oculta, Salida   (Input, Hidden, Output).

In [4]:
def arquitectura(X, Y):
    # nodos de la capa de entrada
    n_x = X.shape[0] 
    # nodos de la capa oculta
    n_h = 10          
    # nodos de la capa de salida
    n_y = Y.shape[0] 
    return (n_x, n_h, n_y)

**2.4 Se definen los Parámetros de la Red Neuronal** 

Los parámetros de una red neuronal son los pesos y los sesgos que se necesitan inicializar a cero. La primera capa solo contiene entradas por lo que no hay pesos ni sesgo, pero tanto las capas ocultas como la capa de salida tienen términos de sesgo y pesos. (W1, b1 y W2, b2)

In [5]:
def parametros(n_x, n_h, n_y):
    W1 = np.random.randn(n_h,n_x) * 0.01     # inicialización aleatoria
    b1 = np.zeros((n_h, 1))                  # inicialización a cero
    W2 = np.random.randn(n_y,n_h) * 0.01 
    b2 = np.zeros((n_y, 1)) 
    return {"W1": W1, "b1": b1, "W2": W2, "b2": b2}    

**2.5 Implementando Forward Propagation**

La capa oculta y la capa de salida se activarán usando la función sigmoide y pasarán la señal hacia adelante. Mientras se computa ésta activación, la entrada se multiplica por el peso y se suma el sesgo antes de pasarla a ésta función.

In [6]:
def forward_propagation(X, params):
    Z1 = np.dot(params['W1'], X)+params['b1']
    A1 = sigmoide(Z1)

    Z2 = np.dot(params['W2'], A1)+params['b2']
    A2 = sigmoide(Z2)
    return {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}    


**2.6 Computando el Error de la Red**

Para computar el coste, una manera directa de hacerlo es calcular el error absoluto entre la predicción y el valor real. Pero una mejor función de pérdida es la función log-pérdida que se define como sigue :

-Sumatoria ( Log (Pred) Real + Log (1 - Pred) Real ) / m


In [7]:
def n_error(Pred, Real):
    logprobs = np.multiply(np.log(Pred), Real)+ np.multiply(np.log(1-Pred), 1-Real)
    coste = -np.sum(logprobs) / Real.shape[1] 
    return np.squeeze(coste)


**2.7 Implementando Backward Propagation**

En la función de backward propagation, el error se pasa hacia atrás a las capas previas  y se calculan las derivadas de los pesos y los sesgos. Entonces se actualizan los pesos y los sesgos a través de las derivadas.


In [8]:
def backward_propagation(params, activacion, X, Y):
    m = X.shape[1]
    
    # capa de salida
    dZ2 = activacion['A2'] - Y                    # calcula derivada del error 
    dW2 = np.dot(dZ2, activacion['A1'].T) / m     # calcula derivada del peso 
    db2 = np.sum(dZ2, axis=1, keepdims=True)/m    # calcula derivada del sesgo
    
    # capa oculta
    dZ1 = np.dot(params['W2'].T, dZ2)*(1-np.power(activacion['A1'], 2))
    dW1 = np.dot(dZ1, X.T)/m
    db1 = np.sum(dZ1, axis=1,keepdims=True)/m
    
    return {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}

def update_parameters(params, derivatives, alpha = 1.2):
    # alpha es la tasa de aprendizaje del modelo 
    
    params['W1'] = params['W1'] - alpha * derivatives['dW1']
    params['b1'] = params['b1'] - alpha * derivatives['db1']
    params['W2'] = params['W2'] - alpha * derivatives['dW2']
    params['b2'] = params['b2'] - alpha * derivatives['db2']
    return params


**2.8 Compila y Entrena el Modelo**

Crea una función que compila todas las funciones clave y crea un modelo de red neuronal.

In [9]:
def neural_network(X, Y, n_h, num_iterations=100):
    n_x = arquitectura(X, Y)[0]
    n_y = arquitectura(X, Y)[2]
    
    params = parametros(n_x, n_h, n_y)
    for i in range(0, num_iterations):
        results = forward_propagation(X, params)
        error = n_error(results['A2'], Y)
        derivatives = backward_propagation(params, results, X, Y) 
        params = update_parameters(params, derivatives)    
    return params

In [10]:
y = Y.values.reshape(1, Y.size)
x = X.T.as_matrix()
model = neural_network(x, y, n_h = 10, num_iterations = 10)

/home/iudh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/home/iudh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


**2.9 Predicciones**

In [11]:
def predice(parameters, X):
    results = forward_propagation(X, parameters)
    print (results['A2'][0])
    predicciones = np.around(results['A2'])    
    return predicciones

predicciones = predice(model, x)
print ('Precisión: %d' % float((np.dot(y,predicciones.T) + np.dot(1-y,1-predicciones.T))/float(y.size)*100) + '%')

[0.61143996 0.07539908 0.95175608 ... 0.95175608 0.07539908 0.95175608]
Precisión: 97%


/home/iudh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


# 3-Implementando una red neuronal, multiclasificación 

En el paso previo se ha discutido sobre como implementar una red neuronal para clasificación binaria. Librerías de Python como sklearn tienen excelentes implementaciones de redes neuronales eficientes que se pueden usar directamente en un dataset. En ésta sección se implementará una red neuronal multiclase para clasificar imágenes de números desde 0 a 9.


**3.1 Preparación del Dataset**

Se separa el dataset en entrenamiento y validación

In [12]:
from sklearn.model_selection import train_test_split
from sklearn import neural_network
from sklearn import  metrics
import tensorflow as tf                           # quitar texto de tensorflow
tf.logging.set_verbosity(tf.logging.ERROR)

Y = train['label'][:10000]                        
X = train.drop(['label'], axis = 1)[:10000]       
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.20, random_state=42)


**3.2 Entrenando el modelo**

Se entrena una red neuronal con 10 capas ocultas.


In [13]:
modelo = neural_network.MLPClassifier(alpha=1e-5, hidden_layer_sizes=(5,), solver='lbfgs', random_state=18)
modelo.fit(x_train, y_train)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=18, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

**3.3 Predicciones**

In [14]:
prediccion = modelo.predict(x_val)
print("Reporte de clasificación:\n %s:" % (metrics.classification_report(y_val, prediccion)))

Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       186
           1       0.96      0.92      0.94       210
           2       0.12      0.99      0.22       220
           3       0.00      0.00      0.00       190
           4       0.00      0.00      0.00       188
           5       0.00      0.00      0.00       194
           6       0.00      0.00      0.00       190
           7       0.00      0.00      0.00       233
           8       0.00      0.00      0.00       197
           9       0.00      0.00      0.00       192

   micro avg       0.20      0.20      0.20      2000
   macro avg       0.11      0.19      0.12      2000
weighted avg       0.11      0.20      0.12      2000
:


/home/iudh/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# 4-DNN-CNN

Una red neuronal profunda (Deep Neural Networks) de un gran número de capas ocultas las cuales tratan de extraer características de bajo nivel de las imágenes. Algunos ejemplor de redesprofundas son las redes convolucionales y las redes neuronales recurrentes.


**Redes Neuronales Convolucionales**

En las redes convolucionales cada imagen de entrada se trata como una matriz de los valores de los pixeles que representan la cantidad de oscuridad en cada pixel. A diferencia de las redes tradicionales que tratan las imágenes como elementos de una dimensión, las redes convolucionales consideran la localización de los pixeles y sus vecinos para la clasificación.

![](neural2.webp)

**Componentes clave de una red convolución**

A. Capa convolucional: en ésta capa, una matriz de pesos (o kernel) se usa para extraer ceracterísticas de bajo nivel de las imágenes. El kernel con sus pesos rota sobre la matriz de la imagen con una ventana corrediza para obtener la salida convolucionada. La matriz de pesos se comporta como un filtro en una imagen extrayendo información particular de la matriz original de la imagen. Durante el proceso de convolución, los pesos se ajustan de tal manera que se minimice la función de pérdida.

B. Stride: El stride se define como el número de pasos que la matriz de pesos da moviéndose por la imagen tomando N pixeles cada vez. Si la matriz de pesos se mueve N pixeles cada vez, se dice que el stride es N.

![](neural3.gif)

Imagen desde.. - www.deeplearning.net

C. Capa Pooling: Las capas de pooling se usan para extraer las características que más información tienen desde la salida convolucional generada.

![](neural4.png)


D. Capa de salida: Para generar la salida final, se aplica una capa densa o completamente conectada (fully connected layer) con una función de activación sigmoide (softmax). La función softmax se usa para generar las probabilidades de cada clase objetivo.




# 5-Implementando una red neuronal convolucional CNN


**5.1 Preparación del Dataset**

En el primer paso se prepara el dataset y se separa en entrenamiento y validación. Para el modelado y el entrenamiento, se usa la librería de Python Keras.



In [15]:
Y = train['label']
X = train.drop(['label'], axis=1)

x_train, x_val, y_train, y_val = train_test_split(X.as_matrix(), Y.as_matrix(), test_size=0.10, random_state=42)


/home/iudh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.



**5.2 Definición de lo parámetros de la red**

Los parámetros son :

Batch    - Número de filas de los datos de entrada para usar en cada iteración para el entrenamiento. 

N Clases - Número de clases objetivo.

Epocas   - Número de iteraciones del modelo de red.


In [16]:
# parametros de la red
batch = 128
n_clases = 10
epocas = 5 

# dimensiones de las imagenes de entrada
img_rows, img_cols = 28, 28


**5.3 Preproceso de la entrada**

En el paso de preprocesamiento, los vectores de la imagen se redimensionan a vectores de 4 dimensiones: batch total, ancho y alto de la imagen y el canal. En este caso el canal es 1, pues es una imagen en blanco y negro y no en (R,G,B). El siguiente paso es normalizar los datos de entrada dividiendo todos los valores entre 255, el máximo del valor de los pixeles.


In [17]:
# preproceso datos entrenamiento 
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_train /= 255

# preproceso datos validacion
x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
x_val = x_val.astype('float32')
x_val /= 255

dim_entrada = (img_rows, img_cols, 1)

# convierte la variable objetivo 
y_train = keras.utils.to_categorical(y_train, n_clases)
y_val = keras.utils.to_categorical(y_val, n_clases)

# preproceso datos test
Xtest = test.as_matrix()
Xtest = Xtest.reshape(Xtest.shape[0], img_rows, img_cols, 1)


/home/iudh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.



**5.4 Creación de la arquitectura del modelo CNN**

En éste paso, se crea la estructura de la red con las siguientes capas:

    Capa convolucional con tamaño de kernel = 3*3, 32 unidades convolutionales, y RelU como función de activación
    
    Capa convolucional con tamaño de kernel = 3*3, 64 unidades convolutional, y RelU como función de activación
    
    Capa Max Pooling Layer con tamaño de pooling = 2*2
    
    Capa de Dropout : Una capa de dropout se usa para regularización y reducir el sobreajuste
    Capa Flatten (Aplana): Una capa para convertir la salida a un array de una dimensión
    
    Capa Densa : Una capa densa es una capa completamente conectada donde cada nodo se conecta a todos los nodos de la siguiente capa. Esta red contiene 128 neuronas pero se puede cambiar para más experimentos
    
    Otra capa Dropout Layer para regularización
    
    Capa Final de salida : Una capa densa con 10 neuronas para generar las clases de salida

En la red más simple que se ha hecho en el paso 1, la función de pérdida era la Log-Loss y el algoritmo de optimización era el Gradiente Descente. En éste caso, se usará la entropía cruzada categórica (categorical_crossentropy), pues es una clasificación multiclase, como función de pérdida y Adadelta como función de optimización.


In [18]:
modelo = Sequential()

# añade primera capa convolucional
modelo.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=dim_entrada))

# añade segunda capa convolucional
modelo.add(Conv2D(64, (3, 3), activation='relu'))

# añade una capa max pooling 
modelo.add(MaxPooling2D(pool_size=(2, 2)))

# añade una capa dropout 
modelo.add(Dropout(0.25))

# añade una capa flatten 
modelo.add(Flatten())

# add dense layer
modelo.add(Dense(128, activation='relu'))

# add another dropout layer
modelo.add(Dropout(0.5))

# add dense layer
modelo.add(Dense(n_clases, activation='softmax'))

# complile the model and view its architecur
modelo.compile(loss=keras.losses.categorical_crossentropy,  optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

modelo.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

5.5 Train the Model

In [19]:
modelo.fit(x_train, y_train, batch_size=batch, epochs=epocas, verbose=1, validation_data=(x_val, y_val))
precision = modelo.evaluate(x_val, y_val, verbose=0)
print('Test de precision:', accuracy[1])

Train on 37800 samples, validate on 4200 samples
Epoch 1/5
37800/37800 [==============================] - 217s 6ms/step - loss: 0.3535 - acc: 0.8905 - val_loss: 0.1042 - val_acc: 0.9717
Epoch 2/5
37800/37800 [==============================] - 218s 6ms/step - loss: 0.1099 - acc: 0.9672 - val_loss: 0.0699 - val_acc: 0.9786
Epoch 3/5
37800/37800 [==============================] - 232s 6ms/step - loss: 0.0791 - acc: 0.9767 - val_loss: 0.0530 - val_acc: 0.9833
Epoch 4/5
37800/37800 [==============================] - 238s 6ms/step - loss: 0.0641 - acc: 0.9811 - val_loss: 0.0500 - val_acc: 0.9843
Epoch 5/5
37800/37800 [==============================] - 235s 6ms/step - loss: 0.0557 - acc: 0.9831 - val_loss: 0.0428 - val_acc: 0.9860
Test accuracy: 0.9859523809523809


5.6 Generate Predictions

In [20]:
pred = modelo.predict(Xtest)
y_clases = pred.argmax(axis=-1)
res = pd.DataFrame()
res['Imagen_Id'] = list(range(1,28001))
res['Etiqueta'] = y_classes
res.to_csv("output.csv", index = False)

Gracias a https://www.kaggle.com/kernels/svzip/4153864